In [1]:
import time 
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import dlib
from imutils import face_utils
# from playsound import playsound 
from pygame import mixer

pygame 2.5.2 (SDL 2.28.3, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Dlib's face detector and the shape predictor for facial landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\shape_predictor_68_face_landmarks.dat')  # Ensure you have this file

# Load your custom model
model = load_model(r'C:\\Users\\virus\\OneDrive\\Desktop\\Proj-Drowsiness-Detection\\models\\best_model.keras')

In [3]:

def get_eye_regions(rects, gray):
    eyes = []
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        # Extract eye regions
        left_eye = shape[36:42]
        right_eye = shape[42:48]
        
        eyes.append(left_eye)
        eyes.append(right_eye)
    
    return eyes



In [4]:
mixer.init()
sound= mixer.Sound(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\alarm.mp3')

In [8]:
import time

cap = cv2.VideoCapture(0)  # You can give direct path

# Initialize variables
eyes_closed_start_time = None
alarm_playing = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)
    eyes = get_eye_regions(rects, gray)

    eyes_closed = False
    for eye in eyes:
        (x, y, w, h) = cv2.boundingRect(np.array(eye))
        eye_img = gray[y:y+h, x:x+w]
        eye_img = cv2.resize(eye_img, (80, 80))  # Resize to 80x80

        # Convert to 3-channel image
        eye_img = np.stack((eye_img,) * 3, axis=-1)
        eye_img = eye_img.astype('float') / 255.0
        eye_img = np.expand_dims(eye_img, axis=0)

        pred = model.predict(eye_img)
        # print(f"Prediction: {pred}")  # Comment out for cleaner output

        if pred[0][0] > pred[0][1]:  # Assuming the first element corresponds to "closed" and the second to "open"
            eyes_closed = True

        for (ex, ey) in eye:
            cv2.circle(frame, (ex, ey), 1, (0, 255, 0), -1)  # Draw circle on detected eye

    # Handle eye closed and open states
    if eyes_closed:
        if eyes_closed_start_time is None:
            eyes_closed_start_time = time.time()  # Start timer when eyes close
    else:
        eyes_closed_start_time = None  # Reset timer when eyes open

    # Play alarm if eyes closed for 10 seconds and not already playing
    if eyes_closed and eyes_closed_start_time is not None and time.time() - eyes_closed_start_time >= 10 and not alarm_playing:
        try:
            sound.play()
            alarm_playing = True  # Set alarm playing flag
        except:
            pass
    elif not eyes_closed:  # Stop alarm when eyes open
        alarm_playing = False  # Reset alarm playing flag

    cv2.imshow('Frame', frame)

    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━